<div id = "monlogo"><center> <img src="https://datascientest.fr/train/assets/logo_datascientest.png", style="height:150px"></center></div>

<hr style="border-width:2px;border-color:#75DFC1"><br>
<center> <h1>Challenge Sorbonne</h1> </center> <br>
<center> <h3>Exemple Modélisation</h3> </center> <br>
<hr style="border-width:2px;border-color:#75DFC1"> 

### Modélisation Machine Learning

>  L'objectif est de créer un modèle permettant d'**identifier pour chaque course de la saison 2021, le vainqueur ainsi que le podium**. Il faut donc créer un modèle de classification binaire, ainsi qu'un modèle de classification multiclasse. Pour rappel, chacun de ces 2 modèles servira pour la moitié de la note de modélisation.
>
> Il est nécessaire de garder ces éléments en tête :
> - **Données futures** : Les Grand Prix ont lieu les uns après les autres, et vous avez dans les différentes tables accès à des données qui n'existent pas encore au moment du départ d'un Grand Prix. Créer un modèle reposant sur des informations futures serait un non-sens mathématiques et une grosse erreur de méthodologie.
> - **Preprocessing** : Il y a bon nombre de données manquantes, incomplètes ou inutiles dans le jeu de données, faites le tri. 
> - **Feature Engineering** : Vous avez un grand nombre de tables, certaines contiennent des features directement exploitables pour le machine learning, d'autres sont intéressantes mais nécessitent des modifications afin d'être utilisées à bon escient. Vous avez également la possibilité d'augmenter vos données à l'aide de librairies comme [FastF1](https://theoehrly.github.io/Fast-F1/), ou en scrapant sur le web.
> - **Métriques et interprétabilités** : Vos modèles seront évaluées selon la métrique bien connue du [f1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html), prenez cela en compte lors de l'optimisation de vos modèles.

### Exemple

> Vous trouverez ci-dessous un exemple de modèle naïf de classification binaire réalisé à partir des tables `results.csv`, `races.csv`, `circuits.csv` et `constructor_standings.csv`, se basant uniquement sur **la position de départ de chaque pilote sur la grille**, **le pays du circuit**, **le classement et le nombre de points constructeur au moment du départ**.
>
> Attention, ce travail de modélisation a uniquement vocation à vous donner une piste pour démarrer, celui-ci n'est pas parfaitement optimisé et ne constitue pas un bon exemple à suivre.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

#Chargement des trois tables
results = pd.read_csv(r"data/results.csv", sep=',')
races = pd.read_csv(r"data/races.csv", sep=',')
circuits = pd.read_csv(r"data/circuits.csv", sep=',')
constructors = pd.read_csv(r"data/constructor_standings.csv", sep=',')

#Jointure entre les différentes tables afin de conserver les 2 features, la variable cible 'positionOrder' et l'année pour effectuer le train_test_split
df1 = pd.merge(results[['driverId', 'constructorId', 'raceId', 'grid', 'positionOrder']], races[['raceId', 'year', 'circuitId']], on = 'raceId')
df2 = pd.merge(df1, constructors[['raceId', 'constructorId', 'position', 'points']], on = ['raceId', 'constructorId'])
df3 = pd.merge(df2, circuits[['circuitId','country']], on = 'circuitId')
df3['winner'] = df3['positionOrder'].apply(lambda x: 1 if x==1 else 0)
df4 = df3.drop(['circuitId', 'constructorId', 'positionOrder'], axis = 1).set_index(['raceId', 'driverId'])

In [2]:
#On encode numériquement les pays
label_encoder = LabelEncoder()
df4.loc[:, 'country'] = label_encoder.fit_transform(df3['country'])
df4.loc[:,"position**2"] = df4["position"] ** 2
df4.loc[:,"points**2"] = df4["points"] ** 2


#On effectue notre train_test_split selon les dates puis on sépare X et y
df_train = df4[df4['year']<=2020]
df_test = df4[df4['year']==2021]

X_train = df_train[['grid', 'country',"points","position", 'points**2',"position**2"]]
X_test = df_test[['grid', 'country', "points","position",'points**2',"position**2"]]

y_train = df_train[['winner']]
#y_test ne contient pas d'informations, puisque c'est ce qu'il faut prédire et soumettre sur la plateforme :)
y_test = df_test[['winner']]
y_pred = df_test[['winner']]

    'max_iter': [3000],
    'penalty' : ["l1", "elasticnet", "l2"],
    'solver' : ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga", "lbfgs"

In [25]:
from sklearn.model_selection import GridSearchCV

# Définition du modèle
Logistic = LogisticRegression(random_state = 42)

# Définition de la grille de paramètres et du GridSearchCV
param_grid = {
    'max_iter': [5000],
    'penalty' : ["l2"],
    'solver' : ["lbfgs"]
}

grid_search = GridSearchCV(Logistic, param_grid, scoring='recall')

# Entraînement du modèle
grid_search.fit(X_train, y_train.values.ravel())
print(grid_search.best_params_)

{'max_iter': 5000, 'penalty': 'l2', 'solver': 'lbfgs'}


{'max_iter': 3500, 'penalty': 'l2', 'solver': 'newton-cg'}
{'max_iter': 3500, 'penalty': 'l1', 'solver': 'liblinear'}
{'max_iter': 4000, 'penalty': 'l1', 'solver': 'liblinear'}
{'max_iter': 4000, 'penalty': 'l2', 'solver': 'lbfgs'}

Best : {'max_iter': 5000, 'penalty': 'l2', 'solver': 'lbfgs'}

In [26]:
grid_search.score(X_train,y_train)

0.0

In [27]:
#On souhaite ajouter un podium fait à partir des probabilités retournées par le modèle random forest

probabilite = grid_search.predict_proba(X_test)
probabilite = pd.DataFrame(probabilite)
probabilite = probabilite.drop(columns=(0))
probabilite = np.array(probabilite)
probabilite
y_pred.loc[:,'PREpositionOrder'] = probabilite
y_pred
# on a donc un classement fait à partir des probabilité qui faut transformer en un classement ordonné du premier au dernier arrivé (1 à 20)

winner  PREpositionOrder
raceId driverId                          
1052   1              0          0.339196
       830            0          0.259787
       822            0          0.273046
       846            0          0.027384
       815            0          0.324157
...                 ...               ...
1073   849            0          0.000122
       841            0          0.000152
       847            0          0.000089
       8              0          0.000044
       853            0          0.000016

[440 rows x 2 columns]

l'objectif est de réaliser un classement pour chaque course

In [28]:
Taille = y_pred.index
raceID = Taille.get_level_values('raceId').unique()

In [30]:
#driverID = y_pred.index.get_level_values('driverId')
PodiumOr0 = []
Top1 = []
for RACE in raceID:
    driverID = y_pred["PREpositionOrder"][RACE].index.get_level_values('driverId')
    array = np.array(y_pred["PREpositionOrder"][RACE])
    ranks = array.argsort()
    ranks = 20 - ranks.argsort()
    count = 0
    for DRIVER in driverID:
        rang = ranks[count]
        if rang <= 3:
            PodiumOr0.append(ranks[count])
            if rang == 1:
                Top1.append(1)
            else:
                Top1.append(0)
        else:
            PodiumOr0.append(0)
            Top1.append(0)
        count += 1 
    count = 0

In [31]:
print(len(PodiumOr0),len(Top1))

440 440


In [32]:
y_pred.loc[:,'winner'] = Top1
y_pred.loc[:,'positionOrder'] = PodiumOr0
y_pred = y_pred.drop(columns=("PREpositionOrder"))

C:\Users\luang\AppData\Local\Temp\ipykernel_6672\1890261867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'positionOrder'] = PodiumOr0


In [33]:
#On exporte notre y_pred au format CSV 
y_pred.to_csv('Logistic_model1OnlyConstructeur.csv')

In [34]:
y_pred

winner  positionOrder
raceId driverId                       
1052   1              1              1
       830            0              0
       822            0              3
       846            0              0
       815            0              2
...                 ...            ...
1073   849            0              0
       841            0              0
       847            0              0
       8              0              0
       853            0              0

[440 rows x 2 columns]

Résultat : 59.8